In [1]:
import numpy as np
import pandas as pd
import regex as re
import os
import bs4
from bs4 import BeautifulSoup

In [2]:
directory = "downloaded"
i = 0
for filename in os.listdir(directory):
    filename2 = filename.replace(".html","")
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        with open(f, "r") as read_file:
            content = read_file.read()

        if not content.__contains__("Learning Objectives"):
            print(filename + " has nothing")
            #move it to the "no objectives" folder
            os.rename(f, f"downloaded/no objectives/{filename}")
            continue

        # if filename2 != "writingforsuccess":
        #     continue

        print("\n\n\n\n\n\n\n\n\n" + filename + " contains Learning Objectives")

        soup = BeautifulSoup(content, 'html.parser')

        # Remove all exercises            
        for el in soup.find_all("div", {"class": "textbox--exercises"}):
            el.decompose()
        for el in soup.find_all("div", {"class": "review-questions"}):
            el.decompose()
        for el in soup.find_all("div", {"class": "critical-thinking"}):
            el.decompose()
        for el in soup.find_all("div", {"class": "exercise"}):
            el.decompose()
        for el in soup.find_all("div", {"title": "Mid-Chapter Questions"}):
            el.decompose()
        for el in soup.find_all("div", {"title": "End of Chapter Questions"}):
            el.decompose()

        # If an element contains "attributions", remove it and the next element
        for el in soup.find_all("h2", text=re.compile("Attributions")):
            el.decompose()
            if el.next_sibling:
                el.next_sibling.decompose()
        for el in soup.find_all("h3", text=re.compile("Attributions")):
            el.decompose()
            if el.next_sibling:
                el.next_sibling.decompose()
        for el in soup.find_all("h3", text=re.compile("Exercises and Critical Thinking")):
            el.decompose()
            if el.next_sibling:
                el.next_sibling.decompose()
        
        for el in soup.find_all("h3", text=re.compile("Supplemental Exercises")):
            el.parent.decompose()

        # If an h3 contains "Section Quiz", remove it and its parent
        for el in soup.find_all("h3", text=re.compile("Section Quiz")):
            el.parent.decompose()

        for el in soup.find_all("h3", text=re.compile("Self-practice")):
            el.parent.decompose()
        for el in soup.find_all("h3", text=re.compile("Self-Practice")):
            el.parent.decompose()


        for el in soup.find_all("strong", text=re.compile("H5P")):
            # print("el.name: " + el.name + " el.parent.name: " + el.parent.name + " el.parent.parent.name: " + el.parent.parent.name)
            if el.parent is not None:
                if el.parent.name != "div":
                    el.parent.parent.decompose()
                else:
                    el.parent.decompose()
            else:
                el.decompose()
            
        for el in soup.find_all("strong", text=re.compile("Practice Exercise")):
            # print("el.name: " + el.name + " el.parent.name: " + el.parent.name + " el.parent.parent.name: " + el.parent.parent.name)
            if el.parent is not None:
                if el.parent.name != "div":
                    el.parent.parent.decompose()
                else:
                    el.parent.decompose()
            else:
                el.decompose()  

        for el in soup.find_all("strong", text=re.compile("Practice EXERCISE")):
            # print("el.name: " + el.name + " el.parent.name: " + el.parent.name + " el.parent.parent.name: " + el.parent.parent.name)
            if el.parent is not None:
                if el.parent.name != "div":
                    el.parent.parent.decompose()
                else:
                    el.parent.decompose()
            else:
                el.decompose()  
                 



        if filename2 == "biology":
            for el in soup.find_all("ol"):
                el.decompose()




        content = soup.prettify()
        
        #remove all \n
        content = re.sub('\n', '', str(content))

        #remove all &nbsp;
        content = re.sub('&nbsp;', '', str(content))

        #remove all the special characters like &#8220;
        content = re.sub(r'&#\d+;', '', str(content))

        #add space after list elements
        content = re.sub('</li>', '</li>\n', str(content))
        
        #remove all the html tags
        content = re.sub('<[^<]+?>', '', str(content))

        #trim each line
        content = re.sub(r'^\s+', '', str(content), flags=re.MULTILINE)            

        #remove all the duplicate spaces
        content = re.sub(r'(\s+)', lambda x:x.group(0), content, flags=re.DOTALL)

        #remove all the broken content
        content = re.sub(r'An interactive H5P element has been excluded from this version of the text. You can view it online here:', '', str(content))

        #remove all the links
        content = re.sub(r'https?://\S+', '', str(content))


        sections = content.split("Learning Objectives")
        lectures=[]
        question_groups=[]
        for (sectionidx,section) in enumerate(sections[1:]):
            #the first few lines are the learning objectives. as soon as a line has one of the following, it is part of the lecture:
            # more than 200 characters
            # a © symbol 
            # it contains the text "Media Attributions"
            # it contains "Figure \d"
            temp = section.split("\n")
            question_group = ""
            lecture = ""
            processing_lec = False
            #in the first line, remove everything that comes before the first colon
            temp[0] = re.sub(r'^.*?:', '', temp[0])
            for t in temp:
                # print("processing: " + t[:200])
                if len(t) > 200 or t.__contains__("©") or t.__contains__("Media Attributions") or re.search(r'Figure \d', t):
                    processing_lec = True
                if processing_lec:
                    lecture += t
                else:
                    question_group += t+"\n"
            
            #trim the question group
            question_group = re.sub(r'^\s+', '', question_group, flags=re.MULTILINE)
            question_group = re.sub(r'\s+$', '', question_group, flags=re.MULTILINE)
            #remove duplicate spaces from the lecture
            lecture = re.sub(r'([ ]+)', ' ', lecture, flags=re.DOTALL)
            #remove duplicate spaces from the question group
            question_group = re.sub(r'([ ]+)', ' ', question_group, flags=re.DOTALL)
            if len(question_group) > 0:
                lectures.append(lecture)
                question_groups.append(question_group)

            # print("question: " + question)
            # print("lecture: " + lecture[:500])
            # break



        print("extracted " + str(len(lectures)) + " lectures and " + str(len(question_groups)) + " questions groups")
        for i in range(min(3, len(lectures))):
            print("QUESTIONS " + str(i))
            print(re.sub(r'\n', ' ### ', question_groups[i][:1000], flags=re.MULTILINE))
            print("LECTURE " + str(i))
            print(re.sub(r'\n', '', lectures[i][:1000], flags=re.MULTILINE))
            print(re.sub(r'\n', '', lectures[i][-1000:], flags=re.MULTILINE))
            print("")



        #this is used to find problems
        if filename2 != "biology":
            blacklist = ["____", "Exercises","Questions","Guided Practice", "Explore More", "Review Questions", "Which of the following", "Test Yourself", "Self-practice"]
            strings = []
            for lecture in lectures:
                if any(lecture.__contains__(word) for word in blacklist):
                    
                    occ = [lecture.find(end) for end in blacklist]
                    idx = np.argmin([999999999 if o == -1 else o for o in occ])
                    loc = occ[idx]
                    strings.append(lecture[loc-50:loc+250])

            if len(strings) > 4:
                print("found too many blacklisted strings")
                for s in strings:
                    print(s)



        if len(lectures) == 1:
            print("only one set of objectives")
            os.rename(f, f"downloaded/no objectives/{filename}")
            continue
        else:
            filename2 = filename.replace(".html","")
            location = f"processed_learning_objectives/{filename2}.pq"
            # create a dataframe that contains all the questions and lectures separately
            df = pd.DataFrame(columns=["lecture", "question_group"])
            for (lecture, question_group) in zip(lectures, question_groups):
                # df = df.append({"lecture": lecture, "question_group": question_group}, ignore_index=True)
                df = pd.concat([df, pd.DataFrame({"lecture": [lecture], "question_group": [question_group]})], ignore_index=True)
                

            # print(df)
            df.to_parquet(location, index=False)
            # break
            continue















biology.html contains Learning Objectives
extracted 100 lectures and 100 questions groups
QUESTIONS 0
Identify and describe the properties of life ### Describe the levels of organization among living things ### List examples of different sub disciplines in biology
LECTURE 0
Watch a video about Evolution by Natural Selection. Biology is the science that studies life. What exactly is life? This may sound like a silly question with an obvious answer, but it is not easy to define life. For example, a branch of biology called virology studies viruses, which exhibit some of the characteristics of living entities but lack others. It turns out that although viruses can attack living organisms, cause diseases, and even reproduce, they do not meet the criteria that biologists use to define life. From its earliest beginnings, biology has wrestled with four questions: What are the shared properties that make something “alive”? How do those various living things function? When faced with t

In [3]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(pd.read_parquet("./processed_learning_objectives/biology.pq"))

                                              lecture  \
0   Watch a video about Evolution by Natural Selec...   
1   Watch a video about the Scientific Method. Fig...   
2   Watch a video about electrons and how the elec...   
3   Watch a video about why we need oxygen and how...   
4   Watch a video about proteins and protein enzym...   
5   Watch a video about eukaryotic cells Watch a v...   
6   Cells fall into one of two broad categories: p...   
7   Watch a video about oxygen in the atmosphere. ...   
8   A cell’s plasma membrane defines the boundary ...   
9   Plasma membranes must allow certain substances...   
10  Active transport mechanisms require the use of...   
11  Watch a video about heterotrophs. Scientists u...   
12  Energy production within a cell involves many ...   
13  The Citric Acid Cycle In eukaryotic cells, the...   
14  In aerobic respiration, the final electron acc...   
15  You have learned about the catabolism of gluco...   
16  All living organisms on ear

In [4]:
merged_df = pd.DataFrame(columns=["lecture", "question_group"])

#go through all the parquet files and merge them
for filename in os.listdir("processed_learning_objectives"):
    if filename.endswith(".pq"):
        df = pd.read_parquet("processed_learning_objectives/" + filename)
        merged_df = merged_df.append(df, ignore_index=True)

merged_df.to_parquet("merged.pq", index=True)

C:\Users\Timothe\AppData\Local\Temp/ipykernel_44424/2000946465.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(df, ignore_index=True)
C:\Users\Timothe\AppData\Local\Temp/ipykernel_44424/2000946465.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(df, ignore_index=True)
C:\Users\Timothe\AppData\Local\Temp/ipykernel_44424/2000946465.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append(df, ignore_index=True)
C:\Users\Timothe\AppData\Local\Temp/ipykernel_44424/2000946465.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  merged_df = merged_df.append

In [10]:
for thing in merged_df.sample(30).itertuples():
    print(thing.lecture)
    print("-----------")
    print(thing.question_group)
    print("--------------------------")

The Citric Acid Cycle In eukaryotic cells, the pyruvate molecules produced at the end of glycolysis are transported into mitochondria , which are sites of cellular respiration. If oxygen is available, aerobic respiration will go forward. In mitochondria, pyruvate will be transformed into a two-carbon acetyl group (by removing a molecule of carbon dioxide) that will be picked up by a carrier compound called coenzyme A (CoA), which is made from vitamin B 5 . The resulting compound is called acetyl CoA. ( Figure 4.17 ). Acetyl CoA can be used in a variety of ways by the cell, but its major function is to deliver the acetyl group derived from pyruvate to the next pathway in glucose catabolism. Figure 4.17 Pyruvate is converted into acetyl-CoA before entering the citric acid cycle. Like the conversion of pyruvate to acetyl CoA, the citric acid cycle in eukaryotic cells takes place in the matrix of the mitochondria . Unlike glycolysis, the citric acid cycle is a closed loop: The last part of